# Day 1-00: 파인튜닝 핵심 개념 빠르게 이해하기

이 노트북은 파인튜닝과 LoRA/QLoRA 개념을 짚고 넘어가는 **개념 정리용** 자료입니다. 실행이 필수인 셀은 없습니다.

### 전체 파인튜닝 vs LoRA vs QLoRA 구조 한눈에 보기
아래 카드를 실행해 구조 차이를 시각적으로 확인하세요.

In [ ]:
from IPython.display import HTML

HTML("""
<style>
  .ft-card-wrapper {display:flex;gap:20px;flex-wrap:wrap;margin:4px 0;}
  .ft-card {flex:1 1 220px;padding:18px;border-radius:14px;border:2px solid #304FFE;background:#E8EAF6;box-shadow:0 6px 18px rgba(48,79,254,0.12);}
  .ft-card h4 {margin:0;font-size:18px;color:#1A237E;}
  .ft-card .tag {display:inline-block;margin-top:10px;padding:4px 10px;border-radius:999px;background:#1A237E;color:white;font-size:12px;letter-spacing:0.3px;}
  .ft-card .detail {margin-top:14px;padding:12px;border-radius:10px;background:white;border:1px solid #C5CAE9;color:#283593;font-size:13px;line-height:1.45;}
</style>
<div class='ft-card-wrapper'>
  <div class='ft-card'>
    <h4>Full Fine-Tuning</h4>
    <span class='tag'>모든 가중치 학습</span>
    <div class='detail'>Base Model FP16 / FP32 전체 파라미터 업데이트<br>GPU · 데이터 요구량 최대</div>
  </div>
  <div class='ft-card'>
    <h4>LoRA</h4>
    <span class='tag'>Base 고정 · 어댑터 학습</span>
    <div class='detail'>Base는 FP16으로 유지하고 LoRA 어댑터만 학습<br>중간 수준 VRAM으로 빠른 실험 가능</div>
  </div>
  <div class='ft-card'>
    <h4>QLoRA</h4>
    <span class='tag'>Base 4bit Quantization</span>
    <div class='detail'>Base를 4bit로 양자화 + LoRA 어댑터 학습<br>Colab T4 등 저자원 환경에서 실행 가능</div>
  </div>
</div>
""")


## 1. 파인튜닝이란?
- **사전 학습(Pre-training)**: 거대한 일반 데이터로 모델이 언어/지식을 학습
- **파인튜닝(Fine-tuning)**: 특정 태스크/도메인 데이터를 더해 세밀하게 조정
- 수식으로는 기존 가중치 $W_0$를 유지한 채 작은 업데이트 $\Delta W$를 학습하는 과정

## 2. 전체 파인튜닝 vs LoRA
| 구분 | 전체 파인튜닝 | LoRA 파인튜닝 |
|------|----------------|----------------|
| 학습 파라미터 | 전체 가중치 (수십억) | 소수 파라미터 (수백~수천만) |
| 메모리 요구량 | 매우 큼 (수십 GB) | 작음 (수 GB 이하) |
| 장점 | 최고 성능 | 빠르고 재사용 쉬움 |
| 단점 | 비용↑, 저장공간↑ | 기본 모델 필요, 표현력 제한 |

LoRA는 큰 행렬을 $B A$ 두 행렬의 곱으로 근사해, 학습해야 할 파라미터를 줄이는 전략입니다.

## 3. 왜 4bit 양자화 + LoRA인가?
1. **4bit Quantization**: 모델 본체를 4bit로 압축 → GPU 메모리 절약
2. **LoRA 어댑터**: 추가로 붙인 작은 행렬만 학습 → 훈련 시간 단축
3. **동작 방식**
   - 원래 가중치 $W_0$는 고정
   - LoRA 어댑터 $B A$만 업데이트
   - 추론 시에는 $W = W_0 + B A$ 형태로 동작, 학습된 어댑터만 교체 가능

## 4. PEFT(파라미터 효율적 파인튜닝) 기법 비교
```text
방법         장점                            고려사항
----------- ------------------------------ --------------------------
Full FT      최고 성능                         비용·시간·저장 모두 큼
LoRA         빠르고 메모리 절약                 기본 모델 필요
Prefix       입력 앞에 벡터 추가               특정 태스크에 최적화 필요
Adapter      중간 레이어에 작은 네트워크 추가   모델 구조 수정 필요
```
LoRA는 적은 변경으로 효과를 얻을 수 있어 가장 널리 사용됩니다.

## 5. QLoRA(4bit + LoRA) 파이프라인 요약
1. **데이터 준비** – 전처리/템플릿 적용 (00.03에서 수행)
2. **모델 준비** – 4bit quantization + LoRA 어댑터 장착 (00.04 참조)
3. **파인튜닝 실행** – main-practice/03 노트북에서 `trainer.train()`
4. **평가** – main-practice/04 또는 00.06에서 지표 확인

## LoRA 성능 체험 (미니 실습)
아래 샘플 데이터를 이용해 **기준 모델 vs LoRA 모델**의 지표 차이를 간단히 계산해 봅니다.

### 파이프라인 이해도 체크 퀴즈
실습에 들어가기 전에 핵심 개념을 빠르게 복습해 봅니다. 각 문항을 선택하고 **정답 확인** 버튼을 눌러보세요.


In [ ]:
import ipywidgets as widgets
from IPython.display import display

questions = [
    {
        "text": "1. QLoRA가 LoRA와 다른 가장 큰 이유는?",
        "options": [
            "LoRA rank를 자동으로 조정한다",
            "Base 모델을 4bit로 양자화해 메모리를 줄인다",
            "학습률을 두 배로 높인다",
        ],
        "answer": "Base 모델을 4bit로 양자화해 메모리를 줄인다",
        "explain": "QLoRA는 기본 모델을 4bit로 양자화한 뒤 LoRA 어댑터만 학습해 VRAM을 크게 줄입니다.",
    },
    {
        "text": "2. LoRA에서 학습 가능한 파라미터 수를 결정하는 요소가 아닌 것은?",
        "options": [
            "LoRA rank (r)",
            "어댑터를 붙이는 레이어 수",
            "평가 지표 종류",
        ],
        "answer": "평가 지표 종류",
        "explain": "평가 지표는 성능 확인용이지 학습 파라미터 수에는 영향을 주지 않습니다.",
    },
    {
        "text": "3. Full Fine-Tuning이 적합한 상황은?",
        "options": [
            "데이터가 적고 신속한 실험이 필요할 때",
            "모델을 완전히 재학습할 만큼 데이터와 자원이 충분할 때",
            "GPU가 6GB 이하일 때",
        ],
        "answer": "모델을 완전히 재학습할 만큼 데이터와 자원이 충분할 때",
        "explain": "모든 가중치를 학습하므로 자원이 넉넉할 때 최대 성능을 기대할 수 있습니다.",
    },
]

feedback_boxes = []

for q in questions:
    dropdown = widgets.Dropdown(options=q["options"], description="선택")
    button = widgets.Button(description="정답 확인", button_style="primary")
    feedback = widgets.HTML(value="")

    def make_handler(dd=dropdown, fb=feedback, answer=q["answer"], explain=q["explain"]):
        def handler(_):
            if dd.value == answer:
                fb.value = f"<span style='color:green;'>✅ 정답! {explain}</span>"
            else:
                fb.value = f"<span style='color:#d84315;'>❌ 다시 생각해보세요. Tip: {explain}</span>"
        return handler

    button.on_click(make_handler())

    display(widgets.VBox([
        widgets.HTML(value=f"<b>{q['text']}</b>"),
        dropdown,
        button,
        feedback,
        widgets.HTML(value="<hr style='margin:8px 0;'>")
    ]))


### LoRA 파라미터 수 빠르게 계산해 보기
아래 계산기로 **숨겨진 차원(hidden size)**, **LoRA rank**, **적용 레이어 수**를 넣으면 학습해야 하는 파라미터 수를 대략적으로 확인할 수 있습니다. (LoRA에서 학습 파라미터 ≈ 2 × Hidden Size × Rank × 적용 레이어 수)


In [ ]:
import ipywidgets as widgets
from IPython.display import display

hidden_size = widgets.IntSlider(value=4096, min=512, max=8192, step=256, description='Hidden')
rank = widgets.IntSlider(value=16, min=4, max=128, step=4, description='Rank')
layer_count = widgets.IntSlider(value=32, min=1, max=64, step=1, description='Layers')

output = widgets.HTML()

def update(*_):
    params = 2 * hidden_size.value * rank.value * layer_count.value
    output.value = f"<b>학습 파라미터 추정:</b> {params:,.0f} 개"

for w in (hidden_size, rank, layer_count):
    w.observe(update, 'value')

update()
display(widgets.VBox([
    widgets.HTML(value="<b>LoRA 파라미터 계산기</b>"),
    hidden_size,
    rank,
    layer_count,
    output,
]))


## 6. LoRA 어댑터와 훈련/추론
- LoRA 어댑터는 **추가 학습을 위해 붙여두는 준비 상태**입니다.
- 기본 모델 + LoRA 어댑터로 추론이 가능하며, 학습이 진행되면 어댑터 가중치만 업데이트 됩니다.
- 학습이 끝난 후에는 `model.save_pretrained()`로 **어댑터만 저장**해 다른 기본 모델에 손쉽게 재사용할 수 있습니다.

## 7. 파인튜닝 vs 프롬프트 엔지니어링 선택 가이드
- 데이터 양/품질이 충분하고 지속적 사용이 필요 → **파인튜닝**
- 빠르게 결과가 필요하거나 모델 수정이 불가능 → **프롬프트 엔지니어링**
- 둘을 병행하면 최상의 결과를 얻을 수 있음 (프롬프트로 기본 품질 확보 → LoRA로 미세 조정)

## 8. 다음 단계로
- 준비가 끝났다면 **main-practice/01 ~ 03**을 순서대로 실행하며 실전 파이프라인을 경험하세요.
- LoRA 맞춤형 어댑터를 학습한 뒤, 00.06 또는 main-practice/04에서 평가 지표를 계산해 성능을 비교합니다.